In [1]:
!pip install PySastrawi
!pip install emoji
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 1.7 MB/s eta 0:00:00a 0:00:01m


In [2]:
import pandas as pd
import numpy as np
import unicodedata
import re
import emoji
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [3]:
# Load the uploaded file into a DataFrame
df = pd.read_csv("/kaggle/input/absa-data/DataABSA - id_data_raw (1).csv")

df.head()

,No.,Review,aspek1,sentimen1,aspek2,sentimen2,aspek3,sentimen3,aspek4,sentimen4,aspek5,sentimen5,aspek6,sentimen6
0,1,"makanannya enak enak, pelayanannya juga mantap...",NaN,NaN,NaN,NaN,NaN,NaN,pelayanan,positif,NaN,NaN,makanan,positif
1,2,"Rasanya ga pernah bisa bohong, setiap ke McD s...",rasa,positif,NaN,NaN,NaN,NaN,NaN,NaN,menu,positif,NaN,NaN
2,3,"Rasanya selalu enak gak pernah berubah, higine...",rasa,positif,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Pelayanannya cukup ramah dan praktis. Makanann...,NaN,NaN,NaN,NaN,NaN,NaN,pelayanan,positif,NaN,NaN,makanan,positif
4,5,"Tempatnya nyaman bersih ,makanannya recommende...",NaN,NaN,tempat,positif,NaN,NaN,NaN,NaN,NaN,NaN,makanan,positif


In [4]:
import pandas as pd

# Membaca data dari file CSV
df = pd.read_csv("/kaggle/input/absa-data/DataABSA - id_data_raw (1).csv")

# List kolom aspek dan sentimen
aspek_cols = [f'aspek{i}' for i in range(1, 7)]
sentimen_cols = [f'sentimen{i}' for i in range(1, 7)]

# Dataframe kosong untuk hasil akhir
result = []

# Loop melalui setiap baris
for index, row in df.iterrows():
    # Loop melalui setiap pasangan aspek dan sentimen
    for aspek_col, sentimen_col in zip(aspek_cols, sentimen_cols):
        aspek = row[aspek_col]
        sentimen = row[sentimen_col]
        if pd.notna(aspek) and pd.notna(sentimen):
            result.append({
                'No.': row['No.'],
                'Review': row['Review'],
                'aspek': aspek,
                'sentimen': sentimen
            })

# Konversi list of dicts ke DataFrame
result_df = pd.DataFrame(result)

# Menampilkan hasil akhir
print(result_df)


       No.                                             Review      aspek  \
0        1  makanannya enak enak, pelayanannya juga mantap...  pelayanan   
1        1  makanannya enak enak, pelayanannya juga mantap...    makanan   
2        2  Rasanya ga pernah bisa bohong, setiap ke McD s...       rasa   
3        2  Rasanya ga pernah bisa bohong, setiap ke McD s...       menu   
4        3  Rasanya selalu enak gak pernah berubah, higine...       rasa   
...    ...                                                ...        ...   
3702  1990  pelayanan buruk banget . Makanan enakk cuma di...    makanan   
3703  1991                   Tempat luar biasa untuk keluarga     tempat   
3704  1992   Suasana nya nyaman,bersih..makanan nya juga enak    makanan   
3705  1993  Tempatnya enak buat makan rame2, keluarga, ram...     tempat   
3706  1993  Tempatnya enak buat makan rame2, keluarga, ram...    makanan   

     sentimen  
0     positif  
1     positif  
2     positif  
3     positif  
4     p

In [5]:
import pandas as pd
import numpy as np
import re
import emoji
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
import unicodedata

class preprocess_data:
    def __init__(self):
        # Load the CSV file into a DataFrame
        df_kamusalay = pd.read_csv('/kaggle/input/absa-data/new_kamusalay.csv', header=None, encoding='latin1')
        self.word_map = dict(zip(df_kamusalay[0], df_kamusalay[1]))

        # Load the abusive words from abusive.csv into a set
        df_abusive = pd.read_csv('/kaggle/input/absa-data/abusive.csv')
        self.abusive_words = set(df_abusive['ABUSIVE'])

        factory = StemmerFactory()
        self.stemmer = factory.create_stemmer()
        stopword_factory = StopWordRemoverFactory()
        self.stopwords = stopword_factory.get_stop_words()

        # Aspect terms to be preserved
        self.aspect_terms = {'aspek', 'pelayanan', 'tempat', 'harga', 'menu', 'rasa', 'makanan'}

        # Remove aspect terms from stopwords
        self.stopwords = [word for word in self.stopwords if word not in self.aspect_terms]

        # Custom stemming dictionary
        self.custom_stem_dict = {
            'makanannya': 'makanan',
            'makanan': 'makanan',
            'pelayanan': 'pelayanan',
            'pelayanannya': 'pelayanan',
            'pelayan': 'pelayanan',
            'layanan': 'pelayanan'
        }

    def case_folding(self, text):
        return text.lower()

    def remove_non_ascii(self, text):
        return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    def remove_punctuation(self, text):
        return re.sub(r'[^\w\s]', ' ', text)

    def remove_repeated_characters(self, text):
        return re.sub(r'\b(\w*?)([^grnlmo])\2+(\w*)\b', r'\1\2\3', text)

    def fix_typos(self, text):
        words = text.split()
        normalized_words = [self.word_map.get(word, word) for word in words]
        return ' '.join(normalized_words)
    
    def remove_abusive_words(self, text):
        words = text.split()
        clean_words = [word for word in words if word.lower() not in self.abusive_words]
        return ' '.join(clean_words)

    def remove_whitespace(self, text):
        return re.sub(r'\s+', ' ', text).strip()

    def emojize(self, text):
        return emoji.demojize(text)

    def tokenize(self, text):
        return word_tokenize(text)

    def remove_stopwords(self, tokens):
        return [word for word in tokens if word not in self.stopwords]

    def stemming(self, text):
        words = text.split()
        stemmed_words = [self.custom_stem_dict.get(word, self.stemmer.stem(word)) for word in words]
        return ' '.join(stemmed_words)
    
    def remove_numbers(self, text):
        return re.sub(r'\d+', '', text)
    
    def preprocess_text(self, text):
        text = self.remove_non_ascii(text)
        text = self.case_folding(text)
        text = self.remove_punctuation(text)
        text = self.remove_repeated_characters(text)
        text = self.fix_typos(text)
        text = self.remove_abusive_words(text)
        text = self.remove_whitespace(text)
        text = self.emojize(text)
        text = self.remove_numbers(text)
        tokens = self.tokenize(text)
        tokens = self.remove_stopwords(tokens)
        cleaned_text = ' '.join(tokens)
        text = self.stemming(cleaned_text)
        return text

In [6]:
cleaned_data = result_df.copy()
cleaned_data = cleaned_data.rename(columns={'Review (lower)': 'Review'})

cleaned_data.head()

,No.,Review,aspek,sentimen
0,1,"makanannya enak enak, pelayanannya juga mantap...",pelayanan,positif
1,1,"makanannya enak enak, pelayanannya juga mantap...",makanan,positif
2,2,"Rasanya ga pernah bisa bohong, setiap ke McD s...",rasa,positif
3,2,"Rasanya ga pernah bisa bohong, setiap ke McD s...",menu,positif
4,3,"Rasanya selalu enak gak pernah berubah, higine...",rasa,positif


In [17]:
!pip install nlpaug
!pip install nltk

In [19]:
import nlpaug.augmenter.word as naw
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import nltk

# Mengunduh corpus WordNet menggunakan NLTK
nltk.download('wordnet')
nltk.download('omw-1.4')

# Mengatur lokasi penyimpanan resource NLTK secara manual (opsional)
import os
nltk.data.path.append('/root/nltk_data')

# Inisialisasi augmenter
aug = naw.SynonymAug(aug_src='wordnet')

# Fungsi untuk melakukan augmentasi teks
def augment_text(text):
    return aug.augment(text)

# Menerapkan augmentasi pada keseluruhan kolom 'Review'
cleaned_data['Augmented_Review'] = cleaned_data['Review'].apply(augment_text)

# Mengubah kolom Augmented_Review ke bentuk string
cleaned_data['Augmented_Review'] = cleaned_data['Augmented_Review'].apply(lambda x: ' '.join(x))

# Menggabungkan data asli dan augmented
augmented_data = cleaned_data.copy()
augmented_data['Review'] = augmented_data['Augmented_Review']
augmented_data = augmented_data.drop(columns=['Augmented_Review'])

final_data = pd.concat([cleaned_data.drop(columns=['Augmented_Review']), augmented_data])

# Melakukan encode aspek
aspect_label_encoder = LabelEncoder()
final_data['aspek'] = aspect_label_encoder.fit_transform(final_data['aspek'])

# Melihat data setelah augmentasi dan preprocessing
print(final_data.head())

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


LookupError: 
**********************************************************************
  Resource 'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/root/nltk_data'
**********************************************************************

In [7]:
# Assuming dn is an instance of the data_normalization class
pd = preprocess_data()

# Apply the normalize_text method to the review column and replace the values
cleaned_data['Review'] = cleaned_data['Review'].apply(lambda x: pd.preprocess_text(x))

cleaned_data.head()

,No.,Review,aspek,sentimen
0,1,makanan pelayanan mantap baik,pelayanan,positif
1,1,makanan pelayanan mantap baik,makanan,positif
2,2,bohong mc donalds ayam spicy mc flurry kadang ...,rasa,positif
3,2,bohong mc donalds ayam spicy mc flurry kadang ...,menu,positif
4,3,ubah higines anak anak suka makan ayam goreng ...,rasa,positif


In [8]:
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

class Config:
    def __init__(self):
        self.batch_size = 32
        self.val_size = 0.1

config = Config()

# Splitting the dataset
train_data, val_data = train_test_split(cleaned_data, test_size=config.val_size, random_state=42)

# Creating DataLoader instances
train_loader = DataLoader(train_data.to_dict('records'), batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_data.to_dict('records'), batch_size=config.batch_size, shuffle=False)

# Print to verify the splits
print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")

train_data.head()

Training set size: 3336
Validation set size: 371


,No.,Review,aspek,sentimen
194,111,mantap penuh bmku pelayanan ekstra ramah proto...,pelayanan,positif
1417,730,salah tempat nongkrong baik makan yakin kualit...,tempat,positif
2459,1255,pariasi makanan sesuai nama restonya darmaga s...,makanan,netral
2293,1173,makanan menu agam buharga jangkau tempat luas ...,makanan,positif
3251,1691,pandang bagus makanan harga jangkau tempat rek...,makanan,positif


In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Assuming cleaned_data is your dataframe
# Convert categorical aspect labels to numerical labels
aspect_label_encoder = LabelEncoder()
cleaned_data['aspek'] = aspect_label_encoder.fit_transform(cleaned_data['aspek'])

# Check the class mapping for aspects
print("Aspect Classes:", aspect_label_encoder.classes_)

# If you still need to convert 'sentimen' column to numerical labels as well
sentiment_label_encoder = LabelEncoder()
cleaned_data['sentimen'] = sentiment_label_encoder.fit_transform(cleaned_data['sentimen'])

# Check the class mapping for sentiments
print("Sentiment Classes:", sentiment_label_encoder.classes_)

cleaned_data.head()


Aspect Classes: ['harga' 'makanan' 'menu' 'pelayanan' 'rasa' 'tempat']
Sentiment Classes: ['negatif' 'netral' 'positif']


,No.,Review,aspek,sentimen
0,1,makanan pelayanan mantap baik,3,2
1,1,makanan pelayanan mantap baik,1,2
2,2,bohong mc donalds ayam spicy mc flurry kadang ...,4,2
3,2,bohong mc donalds ayam spicy mc flurry kadang ...,2,2
4,3,ubah higines anak anak suka makan ayam goreng ...,4,2


In [12]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')

# Prepare the dataset class
class ReviewDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        review = str(self.data.iloc[idx]['Review'])
        aspect = self.data.iloc[idx]['aspek']

        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'aspect': torch.tensor(aspect, dtype=torch.long)
        }

# Load and preprocess the data
# Assuming cleaned_data is already defined as a DataFrame

# Encode aspects
aspect_label_encoder = LabelEncoder()
cleaned_data['aspek'] = aspect_label_encoder.fit_transform(cleaned_data['aspek'])

# Prepare Data
train_data, val_data = train_test_split(cleaned_data, test_size=0.1, random_state=42)
train_dataset = ReviewDataset(train_data, tokenizer, max_len=128)
val_dataset = ReviewDataset(val_data, tokenizer, max_len=128)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize the model
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(aspect_label_encoder.classes_))
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

# Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)


for epoch in range(5):  # 30 epochs
    model.train()
    total_loss = 0
    predictions = []
    true_labels = []

    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        aspect = batch['aspect'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=aspect)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        # Collect predictions and true labels for metrics
        _, preds = torch.max(outputs.logits, dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(aspect.cpu().numpy())

    # Calculate metrics for the current epoch
    predicted_aspects = aspect_label_encoder.inverse_transform(predictions)
    true_aspects = aspect_label_encoder.inverse_transform(true_labels)

    accuracy = accuracy_score(true_aspects, predicted_aspects)
    precision = precision_score(true_aspects, predicted_aspects, average='weighted')
    recall = recall_score(true_aspects, predicted_aspects, average='weighted')
    f1 = f1_score(true_aspects, predicted_aspects, average='weighted')

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# Prediction on validation set
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for batch in tqdm(val_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        true_aspects = batch['aspect'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(true_aspects.cpu().numpy())

# Decode predictions to original aspect labels
predicted_aspects = aspect_label_encoder.inverse_transform(predictions)
true_aspects = aspect_label_encoder.inverse_transform(true_labels)

# Calculate metrics
accuracy = accuracy_score(true_aspects, predicted_aspects)
precision = precision_score(true_aspects, predicted_aspects, average='weighted')
recall = recall_score(true_aspects, predicted_aspects, average='weighted')
f1 = f1_score(true_aspects, predicted_aspects, average='weighted')

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation F1 Score: {f1:.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 105/105 [00:41<00:00,  2.55it/s]


Epoch 1, Loss: 1.2481, Accuracy: 0.4559, Precision: 0.4570, Recall: 0.4559, F1 Score: 0.4440


100%|██████████| 105/105 [00:41<00:00,  2.51it/s]


Epoch 2, Loss: 0.9313, Accuracy: 0.5168, Precision: 0.5187, Recall: 0.5168, F1 Score: 0.5153


100%|██████████| 105/105 [00:41<00:00,  2.50it/s]


...

Epoch 4, Loss: 0.8796, Accuracy: 0.5192, Precision: 0.5200, Recall: 0.5192, F1 Score: 0.5163


100%|██████████| 105/105 [00:41<00:00,  2.50it/s]


Epoch 5, Loss: 0.8580, Accuracy: 0.5207, Precision: 0.5228, Recall: 0.5207, F1 Score: 0.5191


100%|██████████| 12/12 [00:01<00:00,  7.11it/s]

Validation Accuracy: 0.4636
Validation Precision: 0.4370
Validation Recall: 0.4636
Validation F1 Score: 0.4154


In [11]:
import matplotlib.pyplot as plt

train_losses.append(epoch_loss)
train_accuracies.append(epoch_accuracy)


# Plot the training loss and accuracy
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, 31), train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, 31), train_accuracies, label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

NameError: name 'train_losses' is not defined